In [11]:
import numpy as np
import ee
import collections
# collections.Callable = collections.abc.Callable
# ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWiWQTlGm4TImPV55kNhe4N_YbqqPA4hSiafMYn7OU_HxRavUvfRuzo



Successfully saved authorization token.


In [12]:
ex = np.genfromtxt("GEE/gedi/compiled_test_gedi_score.csv", delimiter=',', skip_header=1)

In [13]:
#ex = data[0:10]
# print(ex)

In [14]:
new_array = np.empty((0,13), float)
for row in ex:
    min_lon = row[0]
    min_lat = row[1]
    max_lon = row[2]
    max_lat = row[3]
    #nasa = row[9]
    gedi = row[10]
    #elev_loss = row[11]
    score = 0
    
    right_index = np.where((ex[:, 0] == max_lon) & (ex[:, 1] == min_lat) & (ex[:, 3] == max_lat))
    left_index = np.where((ex[:, 1] == min_lat) & (ex[:, 2] == min_lon) & (ex[:, 3] == max_lat))
    up_index = np.where((ex[:, 0] == min_lon) & (ex[:, 1] == max_lat) & (ex[:, 2] == max_lon))
    down_index = np.where((ex[:, 0] == min_lon) & (ex[:, 2] == max_lon) & (ex[:, 3] == min_lat))
    up_right_index = np.where((ex[:, 0] == max_lon) & (ex[:, 1] == max_lat))
    up_left_index = np.where((ex[:, 1] == max_lat) & (ex[:, 2] == min_lon))
    down_right_index = np.where((ex[:, 0] == max_lon) & (ex[:, 3] == min_lat))
    down_left_index = np.where((ex[:, 2] == min_lon) & (ex[:, 3] == min_lat))
    
    #just adds a column with number of neighbors with gedi elevation < 0 - not actually doing anything useful lol but it works
    right_gedi = ex[(right_index[0]),10] 
    left_gedi = ex[(left_index[0]),10]   
    up_gedi = ex[(up_index[0]),10] 
    down_gedi = ex[(down_index[0]),10] 
    up_right_gedi = ex[(up_right_index[0]),10] 
    up_left_gedi = ex[(up_left_index[0]),10]   
    down_right_gedi = ex[(down_right_index[0]),10] 
    down_left_gedi = ex[(down_left_index[0]),10] 
    
    neighbors = [right_gedi, left_gedi, up_gedi, down_gedi, up_right_gedi, up_left_gedi, down_right_gedi, down_left_gedi]
    for x in neighbors:
        if x.size > 0:
            #previous: x<0 and ...
            if (gedi < x and x != -999):
                score = score+1
    new_row = np.append(row, score, axis=None)
    new_array = np.append(new_array, np.array([new_row]), axis=0)

In [15]:
# Convert to a Feature Collection
# Create a list of Features for areas, with score as property

region_list = []
for row in new_array:
    region = ee.Geometry.Polygon([[[float(row[0]), float(row[3])],
                                  [float(row[0]), float(row[1])],
                                  [float(row[2]), float(row[1])],
                                  [float(row[2]), float(row[3])]]])
    f = ee.Feature(region).set('elevation score', float(row[12]))
    region_list.append(f)

# Wrap feature list in a Feature Collection
fc = ee.FeatureCollection(region_list)

# Export the Feature Collection to Google Earth Engine (GEE)
task = ee.batch.Export.table.toAsset(**{
  'collection': fc,
  'description':'compiled_results',
  'assetId': 'users/EmilyNason/compiledResultsGediFixed', # change to your GEE Asset path and a unique name (will not overwrite already existing assets, so old names cannot be reused)
});

task.start()